In [19]:
import random
import numpy as np
from math import sqrt
from collections import defaultdict
from time import time

In [20]:
def ParseDataFromFile(f):
    for l in open(f):
        yield eval(l)
data=list(ParseDataFromFile("train.json"))
np.random.shuffle(data) 

In [21]:
def load(data):
    for i in data:
        UserS.add(i['userID'])
        BuzS.add(i['businessID'])

    j=0
    for i in UserS:
        UserS2N[i]=j
        UserN2S[j]=i
        j+=1

    j=0
    for i in BuzS:
        BuzS2N[i]=j
        BuzN2S[j]=i
        j+=1
UserS   = set()
UserS2N = {}
UserN2S = {}
BuzS    = set()
BuzS2N  = {}
BuzN2S  = {}
load(data)
lenu=len(UserS)
lenb=len(BuzS)

In [22]:
train=data[:166667]
validate=data[166667:]

UserBlist_T=defaultdict(list);BuzUlist_T =defaultdict(list)
for i in train:
    u=UserS2N[i['userID']];b=BuzS2N[i['businessID']]
    UserBlist_T[u].append(b);BuzUlist_T[b].append(u)
    
UserBlist=defaultdict(list);BuzUlist =defaultdict(list)
for i in data:
    u=UserS2N[i['userID']];b=BuzS2N[i['businessID']]
    UserBlist[u].append(b);BuzUlist[b].append(u)
    
xy_V={}
for i in validate:
    u=UserS2N[i['userID']];b=BuzS2N[i['businessID']]
    xy_V[(u,b)]=1

i=0
while i<len(validate):
    b=int(random.uniform(0,len(BuzS)))
    u=int(random.uniform(0,len(UserS)))
    if b not in UserBlist[u] and (u,b) not in xy_V:
        xy_V[(u,b)]=0;i+=1

In [23]:
xy={}
for i in data:
    u=UserS2N[i['userID']];b=BuzS2N[i['businessID']]
    xy[(u,b)]=1

i=0
while i<len(data):
    b=int(random.uniform(0,len(BuzS)))
    u=int(random.uniform(0,len(UserS)))
    if b not in UserBlist[u] and (u,b) not in xy:
        xy[(u,b)]=0;i+=1

In [24]:
Buzt_T={}
for i in BuzUlist_T:
    Buzt_T[i]=len(BuzUlist_T[i])

In [26]:
class BPR(object):
    def __init__(self,trainpairs,testset, F=10, alpha=0.03, lmbd={'u':0.1,'p':0.1,'n':0.1}, max_iter=500):
        '''
        lmbd是字典{u:1,p:1,n:1}
        F维度
        trainpairs: u:[i0,i1]
        '''
        self.F = F
        self.gu=np.matrix([[random.random() for i in range(F)] for i in range(lenu)])
        self.gi=np.matrix([[random.random() for i in range(F)] for i in range(lenb)])
        self.alpha = alpha
        self.lmbd = lmbd
        self.max_iter = max_iter
        self.trainpairs = trainpairs
        self.testset=testset
        #self.coeffp=0
        self.tmp={}
    def sample(self):
        u=int(random.uniform(0,lenu))
        while u not in self.trainpairs:
            u=int(random.uniform(0,lenu))
        i=np.random.choice(self.trainpairs[u])
        j=i
        while j in self.trainpairs[u]:
            j=int(random.uniform(0,lenb))
        return u,i,j
    
    def update(self,u,i,j):
        #assert u in self.trainpairs and isinstance(i,int) and isinstance(j,int)," sample wrong"
        #assert i!=j and i in self.trainpairs[u] and j not in self.trainpairs[u]," sample wrong"
        x_ui=float(np.inner(self.gu[u],self.gi[i]))
        x_uj=float(np.inner(self.gu[u],self.gi[j]))
        x_uij=x_ui-x_uj
#         if x_uij>10:
#             return
        self.tmp['u']=u;self.tmp['i']=i;self.tmp['j']=j;self.tmp['x_ui']=x_ui;self.tmp['x_uj']=x_uj;self.tmp['x_uij']=x_uij#
        
        #coeff=np.e ** (-x_uij) / (1 + np.e ** (-x_uij))
        coeff=1 / (1 + np.e ** (x_uij))
        #self.coeffp=coeff
        gradu= self.gi[i]-self.gi[j]
        nextgu=self.alpha*(coeff*gradu+self.lmbd['u']*self.gu[u])#/Buzt_T[i]##？
        gradi= self.gu[u]
        nextgi=self.alpha*(coeff*gradi+self.lmbd['p']*self.gi[i])#/Buzt_T[i]##？
        gradj=-self.gu[u]
        nextgj=self.alpha*(coeff*gradj+self.lmbd['n']*self.gi[j])#/Buzt_T[i]##？
        
        self.gu[u]+=nextgu
        self.gi[i]+=nextgi
        self.gi[j]+=nextgj
        
    def train(self,inputt=-1):
        '''随机梯度下降法训练参数gu和gi '''
        t=self.max_iter if inputt<1 else inputt
        for step in xrange(t):

            u,i,j=self.sample()
            self.update(u,i,j)
        #self.alpha*=0.9   
        
    
    def test1(self):
        err=0
        for pair in self.testset:
            u=pair[0];i=pair[1];l=self.testset[pair]*2-1
            err+=np.inner(self.gu[u],self.gi[i])*l
            #err+=(tmp+l)%2
        #err/=1.0*len(self.testset)
        return err

In [242]:
model=UserBlist,xy_V,F=20, alpha=0.1, lmbd={'u':-0.1,'p':-0.2,'n':-0.1}, max_iter=10000)

In [253]:
#tttt=0
model.alpha=1

In [281]:
model.alpha*=0.5

In [282]:
for ll in range(10):
    
    model.train(63333)
    tttt=tttt1
    
    tttt1=float(model.test1())    
    print model.alpha,'\t',tttt1
    if tttt1<tttt:
        print "decrease, stop"
        #break

for ll in range(10):    
    model.train(633333)
    tttt=tttt1
    
    tttt1=float(model.test1())    
    print model.alpha,'\t',tttt1
    if tttt1<tttt:
        print "decrease, stop"
        #break

for ll in range(10):    
    model.train(633333)
    tttt=tttt1
    
    tttt1=float(model.test1())    
    print model.alpha,'\t',tttt1
    if tttt1<tttt:
        print "decrease, stop"
        #break

0.015625 	42309.2288346
0.015625 	42301.2609074
decrease, stop
0.015625 	42303.933012
0.015625 	42306.4617968
0.015625 	42299.9863614
decrease, stop
0.015625 	42300.4340457
0.015625 	42305.3157069
0.015625 	42307.7146347
0.015625 	42309.2580255
0.015625 	42312.603106
0.015625 	42314.1499198
0.015625 	42308.3922325
decrease, stop
0.015625 	42313.7172852
0.015625 	42325.4606974
0.015625 	42338.7827184
0.015625 	42347.8442181
0.015625 	42342.9543548
decrease, stop
0.015625 	42335.5222933
decrease, stop
0.015625 	42339.8654849
0.015625 	42339.745182
decrease, stop
0.015625 	42352.5598476
0.015625 	42363.6494724
0.015625 	42318.8936925
decrease, stop
0.015625 	42329.5356164
0.015625 	42344.5513287
0.015625 	42354.1666078
0.015625 	42339.3232473
decrease, stop
0.015625 	42346.8072746
0.015625 	42371.0417459
0.015625 	42365.5363056
decrease, stop


In [283]:
testacu()

0.9188275


In [287]:
def distance1(ds,b1,b2):#????
    set1=set(ds[b1])
    set2=set(ds[b2])
    try:
        return len(set1 & set2)*1.0/ sqrt(len(set1)) /sqrt(len(set2))
    except:
        return 0

In [298]:
d0t=[];dlfm=[]
#d1t=[]
yt =[]
for i in xy_V:
    dd0=0;dd1=0;dd2=0
    for b2 in UserBlist_T[i[0]]:
        dd0+=distance1(BuzUlist_T,b2,i[1])
    tmp=float(np.inner(model.gu[i[0]],model.gi[i[1]]))
    d0t.append(dd0)#user距离
    dlfm.append(tmp)
    #d1t.append(dd0*1.0/pu[i[0]])#user距离除以买的东西数量
    yt.append(xy_V[i])

In [302]:
findmid=[]
for (u,b) in xy_V:
    findmid.append(float(np.inner(model.gu[u],model.gi[b])))
findmid.sort()
midian=findmid[33333];print midian#(40000-63)/2#-1;

0.63870512412


In [304]:
acc=0
for i in range(len(dlfm)):
    if dlfm[i]<midian and yt[i]==0:
        acc+=1

    elif dlfm[i]>=midian and yt[i]==1:
        acc+=1
print acc*1.0/(len(dlfm))

0.920109201092


In [300]:
for amp in [1,10,30,40,41,42,43,50,60,100,1000]:
    out=1- sum([((d0t[i]*amp>1)+yt[i])%2 for i in range(len(d0t))])*1.0/len(d0t)
    print amp,'\t',out

1 	0.605196051961
10 	0.827618276183
30 	0.850973509735
40 	0.851858518585
41 	0.851963519635
42 	0.852023520235
43 	0.852083520835
50 	0.852158521585
60 	0.852188521885
100 	0.852203522035
1000 	0.852203522035


In [305]:
for amp in [1,10,30,40,41,42,43,50,60,100,1000]:
    out=1- sum([((d0t[i]*amp>1 or dlfm[i]>midian)+yt[i])%2 for i in range(len(d0t))])*1.0/len(d0t)
    print amp,'\t',out

1 	0.928194281943
10 	0.936189361894
30 	0.930969309693
40 	0.930384303843
41 	0.930384303843
42 	0.930384303843
43 	0.930399303993
50 	0.930339303393
60 	0.930249302493
100 	0.930189301893
1000 	0.930189301893


In [252]:
model.tmp

{'i': 7345,
 'j': 4494,
 'u': 1070,
 'x_ui': 1.4449946160492417,
 'x_uij': 1.9953248916334951,
 'x_uj': -0.5503302755842535}

In [164]:
#model.gu[model.tmp['u']],model.gi[model.tmp['i']]

In [257]:
def testacu():
    a=[[],[]]
    for pair in xy:
        u=pair[0];i=pair[1];
        a[xy[pair]].append(float(np.inner(model.gu[u],model.gi[i])))

    a01=[]
    for i in a[0]:
        a01.append(i)
    for i in a[1]:
        a01.append(i)
    a01.sort()

    acc=0
    for i in a[0]:
        if i<a01[len(a[0])]:
            acc+=1
    for i in a[1]:
        if i>a01[len(a[0])]:
            acc+=1
    print acc*1.0/(len(a[0])+len(a[1]))

In [ ]:
#0.9169075 

In [267]:
sum(a[0])/len(a[0]),sum(a[1])/len(a[1]),len(a[0]),len(a[1])

(1.1585785151980366, 31.540506955866753, 200000, 200000)

In [347]:
findmid=[]
for (u,b) in test:
    findmid.append(float(np.inner(model.gu[u],model.gi[b])))

findmid.sort()
midian=findmid[19968+432];print midian#(40000-63)/2#-1;#19968

0.509728664961


In [348]:
# for i in range(len(findmid)):
#     if 0.457975016921<findmid[i]<=0.457975016922:
#         print i

In [321]:
predictions = open("predictions_Visiting.txt", 'w')
#noUorB=0
#findone=0
one_c=0
zero_c=0;cdd0=0
findmid=[]
count=0
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    predictions.write(u + '-' + b + ',')
    
    count+=1
    if count <5000:###############################
        predictions.write(str(0)+ '\n')
        continue
    
    if u not in UserS2N or b not in BuzS2N:
        #noUorB+=1
        predictions.write(str(0) + '\n')        
    else:
        u = UserS2N[u]
        b = BuzS2N[b]
        if b in UserBlist[u]:#11
            #print "fuck"
            predictions.write('1'+'\n')
            continue
            
            
        flag = 0
        p=float(np.inner(model.gu[u],model.gi[b]))
        if p>midian :
        #if p>midian:
            flag=1
            one_c+=1
        if p<=midian:
            flag=0
            zero_c+=1
        
        predictions.write(str(flag) + '\n')
print cdd0
print one_c
print zero_c
predictions.close()
#print noUorB#63

0
17473
17458


In [447]:
findmid=[]
for (u,b) in test:
    findmid.append(float(np.inner(model.gu[u],model.gi[b])))

findmid.sort()
midian=findmid[19968+720];print midian#(40000-63)/2#-1;
low=findmid[5000];print low
high=findmid[25000];print high

0.541991444599
-0.264852650889
0.948936078318


In [448]:
predictions = open("predictions_Visiting.txt", 'w')
#noUorB=0
#findone=0
one_c=0
zero_c=0;cdd0=0
findmid=[]
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    predictions.write(u + '-' + b + ',')
    
    
    if u not in UserS2N or b not in BuzS2N:
        #noUorB+=1
        predictions.write(str(0) + '\n')        
    else:
        u = UserS2N[u]
        b = BuzS2N[b]
        if b in UserBlist[u]:#11
            #print "fuck"
            predictions.write('1'+'\n')
            continue
        flag = 0
        
        
        dd0=0
        for b2 in UserBlist[u]:
            dd0+=distance1(BuzUlist,b2,b)
        
        
        p=float(np.inner(model.gu[u],model.gi[b]))
        
        if p<=midian:
            #if dd0*9>1:
            #    cdd0+=1
            flag=0
            zero_c+=1
        if midian>=p>low and dd0*9>1:
            cdd0+=1
            flag=1
            one_c+=1
        if p>midian:
        #if p>midian or dd0*9>1:
            flag=1
            one_c+=1
#             if len(BuzUlist[b])<3 and len(UserBlist[u])<3 and p<high:
#                 cdd0+=1
#                 #print p
#                 flag=0
#                 one_c-=1
#                 zero_c+=1
        predictions.write(str(flag) + '\n')
print cdd0
print one_c
print zero_c
predictions.close()
#print noUorB#63

1546
20783
20689


In [ ]:
high u 1 
25000->28 18268.0 没用
    i 1
25000->41 18259.0 没用



low
5000->1554 18277.0
8000->1524  18273
10000->1480 18269.0
12000->1426 18265.0

In [434]:
0.91295*2e4

18259.0

## build test

In [ ]:
noUorB=0
findone=0
test=[]
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        continue
    u,b = l.strip().split('-')
    if u not in UserS2N or b not in BuzS2N:
        noUorB+=1
        continue     
    else:
        u = UserS2N[u]
        b = BuzS2N[b]
        if b in UserBlist[u]:
            #print "fuck"
            findone+=1
            test.append((u,b))
            continue
        test.append((u,b))
print findone
#print noUorB#63

In [392]:
?


In [ ]:
print model.alpha
model.tmp

In [636]:
model.gu[model.tmp['u']],model.gi[model.tmp['i']]

(matrix([[ -8.89192839,  -0.73151304,  -4.50616416,   0.89096642,
            6.50655107,  -7.09088872,  -4.24004883,   6.28662233,
            3.01824697,  -6.74725937,  -0.54622962,  -3.18068238,
          -17.73218477,  -3.07281945,  -7.04074195,   3.06614387,
            6.0312796 ,  -3.59311379,   0.86464865,   3.70170105]]),
 matrix([[ -8.81440162,   0.55277196,  -4.1964162 ,   1.34371342,
            5.9010913 ,  -7.09807532,  -7.71433164,   5.58618331,
            3.25524975,  -6.19698867,  -1.5872698 ,  -2.2408341 ,
          -16.17759318,  -1.44255571,  -6.83886146,   2.95858293,
            7.62228204,  -2.17230431,  -0.68162792,   3.3671223 ]]))

{'i': 1527,
 'j': 1025,
 'u': 6098,
 'x_ui': 728.4376729783028,
 'x_uij': 764.9531973130106,
 'x_uj': -36.515524334707806}

In [506]:
BuzUlist[14246]

[8845, 11593]

In [309]:
c=0
for i in BuzUlist_T:
    c+=len(BuzUlist_T[i])
    if len(BuzUlist_T[i])>160:
        print i
print c*1.0/len(BuzUlist_T)

13998
7.83946378175


In [304]:
len(BuzUlist_T)

21260

In [305]:
len(UserBlist_T)

18751

In [639]:
np.matrix([[(-1+2*random.random())/sqrt(10) for i in range(10)] for i in range(3)])

matrix([[-0.06322599,  0.28266666,  0.173661  ,  0.11295225,  0.08433847,
          0.17080739, -0.28935086,  0.13841665,  0.23825169, -0.26486144],
        [ 0.09578499,  0.17602835, -0.22739015,  0.07135965, -0.17681224,
          0.01254262,  0.08640948,  0.14579062, -0.06488709, -0.20864786],
        [-0.07856159, -0.20942394,  0.00287907,  0.18783359, -0.30029122,
          0.28321223, -0.21056918,  0.06507283,  0.19593247, -0.01972849]])